In [1]:
import os 
os.chdir('../')
%pwd

'c:\\Users\\anuru\\Desktop\\MyPro\\Text-Summarizer'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:Path
    data_path:Path
    model_path:Path
    tokenizer_path:Path
    metric_file_name:Path

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [4]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config=self.config.model_evaluation
        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path),
            model_path=Path(config.model_path),
            tokenizer_path=Path(config.tokenizer_path),
            metric_file_name=Path(config.metric_file_name)
        )

        return model_evaluation_config

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import evaluate
import torch 
import pandas as pd
from tqdm import tqdm 

In [ ]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config=config
    
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i:i+batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer, batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", column_text="article", column_summary="highlights"):
        article_batches=list(self.generate_batch_sized_chunks(dataset[column_text]), batch_size)
        target_batches=list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm( zip(article_batches,target_batches), total=len(article_batches)):
            inputs=tokenizer(article_batch, max_length=1024, truncation=True, padding="max_lenght", return_tensors="pt")
            summaries=model.generate(input_ids=inputs["input_ids"].to(device),
                                     attention_mask=inputs["attentions_mask"].to(device),
                                     length_penalty=0.8, num_beams=8, max_lenght=128)
            decoded_summaries=[tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
            decoded_summaries=[d.replace("", " ") for d in decoded_summaries]
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            score=metric.compute()
            return score


    def evaluation(self):
        device="cuda" if torch.cuda.is_available() else "cpu"
        tokenizer=AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        dataset_samsum_pt=load_from_disk(self.config.data_path)
        rouge_names=["rouge1", "rouge2","rougeL","rougeLsum"]
        rouge_metric=evaluate('rouge')

        score=self.calculate_metric_on_test_ds(dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer,batch_size=2, column_text='dialogue', column_summary='summary')
        rouge_dict=dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
        df=pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name,index=False)

In [ ]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(model_evaluation_config)
    model_evaluation.evaluation()
except Exception as e:
    raise e